### 5) Final code (JIT version)

In [28]:
import numpy as np
from __future__ import division
import math
import time

####Initial simulated data

In [29]:
np.random.seed(1)

A=np.array([[0,1,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
   [0,0,0,1,1,1,0,0,0,1,0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0,0],
   [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,1,0]])

num_objects=100
object_dim=36
 
sigma_x_orig=0.5

I=sigma_x_orig*np.eye(object_dim)
Z_orig=np.zeros((num_objects,4))

X=np.zeros((num_objects,object_dim))

for i in range(num_objects):
    Z_orig[i,:]=(np.random.uniform(0,1,4)>0.5)
    while sum(Z_orig[i,:])==0:
        Z_orig[i,:]=(np.random.uniform(0,1,4)>0.5)
    X[i,:]=np.dot(np.random.normal(0,1,object_dim),I)+np.dot(Z_orig[i,:],A)

In [30]:
X

array([[-0.26408588, -0.53648431,  0.43270381, ..., -0.0063323 ,
        -0.55865517,  0.11720785],
       [-0.37357915,  1.8462273 ,  0.02540388, ..., -1.01110061,
        -0.15310201,  0.41398732],
       [-0.11116407,  0.89962097,  0.0932807 , ..., -0.42975797,
         1.17527299, -0.65614171],
       ..., 
       [-0.34608745, -0.57328763,  0.2839653 , ..., -0.15453644,
         0.85995226, -0.39341796],
       [ 0.9069873 , -0.42860736, -0.57521005, ..., -0.32568077,
        -0.02485471,  0.03989475],
       [-0.82683494,  0.48477701,  0.42043053, ...,  0.73417024,
         1.32870971, -0.29110863]])

####IBP prior

In [31]:
%%file jit_functions.py
from numba import jit
import numpy as np

@jit

def likelihood(X, Z, M, sigma_A, sigma_X, K_plus, num_objects, object_dim):
    log_ll=(-1)*num_objects*object_dim*0.5*np.log(2*np.pi)-1*(num_objects-K_plus)*object_dim*np.log(sigma_X)-K_plus*object_dim*np.log(sigma_A)-object_dim*(0.5)*np.log(np.linalg.det((np.dot(Z.T,Z) + np.dot((sigma_X**2/sigma_A**2),np.eye(K_plus)))))+(-1/(2*sigma_X**2))*np.trace(np.dot(np.dot(X.T,(np.eye(num_objects)-np.dot(np.dot(Z,M),Z.T))),X))
    return log_ll

def sampleIBP(alpha, num_objects):
    result=np.zeros((num_objects, 1000))
    t=np.random.poisson(alpha)
    result[0,0:t]=np.ones(t)
    K_plus=t
    p=np.array((0,0))
    for i in range(2, num_objects+1):
        for j in range(K_plus):
            p[0]=np.log(sum(result[0:i,j]))-np.log(i) #doubt in indices
            p[1]=np.log(i - sum(result[0:i,j])) - np.log(i)
            p = np.exp(p-max(p))
            if np.random.uniform(0,1,1)<(p[0]/sum(p)):
                result[i-1,j]=1
            else:
                result[i-1,j]=0
        t=np.random.poisson(alpha/i)
        result[i-1,K_plus:K_plus+t]=np.ones(t) #doubt in indices
        K_plus=K_plus+t
    
    result=result[:,0:K_plus]
    return(result,K_plus)

Overwriting jit_functions.py


####Gibbs Sampler and MH (with timing)

In [34]:
import numpy as np
import math
import time
import jit_functions as func

def sampler(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects):
    HN=0.0
    for i in range(1,num_objects+1): #check indices
        HN=HN+1.0/i

    K_inf=20

    sam=func.sampleIBP(alpha,num_objects)

    Z=sam[0]
    K_plus=sam[1]

    chain_Z=np.zeros((SAMPLE_SIZE,num_objects,K_inf))
    chain_K=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_X=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_A=np.zeros((SAMPLE_SIZE,1))
    chain_alpha=np.zeros((SAMPLE_SIZE,1))

    s_counter=0
    for e in range(E):
        #print e, K_plus, alpha
        if((e+1)>BURN_IN):
            chain_Z[s_counter,:,0:K_plus]=Z[:,0:K_plus]
            chain_K[s_counter]=K_plus
            chain_sigma_X[s_counter]=sigma_X
            chain_sigma_A[s_counter]=sigma_A
            chain_alpha[s_counter]=alpha
            s_counter=s_counter+1

        for i in range(num_objects):
            #M=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
            for k in range(K_plus):
                if (k+1)>K_plus: #doubt
                    break
                if Z[i,k]>0:
                    if (sum(Z[:,k]) - Z[i,k])<=0:
                        Z[i,k]=0
                        Z[:,k:(K_plus-1)] = Z[:,(k+1):K_plus] #doubt in indices
                        K_plus = K_plus-1
                        #Z[:,K_plus]=0
                        #M=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
                        continue
                #M1 = calcInverse(Z[:,0:K_plus], M, i, k, 1) #some shit
                #M2 = calcInverse(Z[:,0:K_plus], M, i, k, 0) #some shit

                P=np.array([0,0])

                Z[i,k]=1
                M1=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
                P[0]=func.likelihood(X, Z[:,0:K_plus], M1, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(sum(Z[:,k])- Z[i,k]) -np.log(num_objects)

                Z[i,k]=0
                M2=np.linalg.inv((np.dot(Z[:,0:K_plus].T,Z[:,0:K_plus]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus))))
                P[1]=func.likelihood(X, Z[:,0:K_plus], M2, sigma_A, sigma_X, K_plus, num_objects, object_dim) + np.log(num_objects - sum(Z[:,k])) -np.log(num_objects)

                P=np.exp(P - max(P))

                if np.random.uniform(0,1,1)<(P[0]/(P[0]+P[1])):
                    Z[i,k] = 1
                    M = M1
                else:
                    Z[i,k] = 0
                    M = M2

            trun=np.zeros(5) #try 4
            alpha_N = alpha/num_objects

            for k_i in range(5):
                if Z.shape[1]>(K_plus+k_i):
                    #Z[:,K_plus:(K_plus+k_i)]=0 #added later
                    Z[i,K_plus:(K_plus+k_i)]=1       
                else:
                    Ztemp=np.zeros((Z.shape[0],K_plus+k_i))
                    Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
                    #Ztemp[:,K_plus:(K_plus+k_i)] = 0 #added later
                    Ztemp[i,K_plus:(K_plus+k_i)] = 1
                    Z=Ztemp
                #doubt in indices
                M=np.linalg.inv((np.dot(Z[:,0:(K_plus+k_i)].T,Z[:,0:(K_plus+k_i)]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye((K_plus+k_i)))))
                trun[k_i] = k_i*np.log(alpha_N) - alpha_N - np.log(math.factorial(k_i)) + func.likelihood(X, Z[:,0:(K_plus+k_i)], M, sigma_A, sigma_X, K_plus+k_i, num_objects, object_dim)

            Z[i,K_plus:K_plus+4] = 0 #check indices
            trun = np.exp(trun - max(trun))
            trun = trun/sum(trun)
            p = np.random.uniform(0,1,1)
            t = 0
            #new_dishes=0
            for k_i in range(5):
                t = t+trun[k_i]
                if p<t:
                    new_dishes = k_i
                    break
            if Z.shape[1]>(K_plus+new_dishes):
                Ztemp=Z
                #Ztemp[:,K_plus:(K_plus+new_dishes)]=0 #added later
                Ztemp[i,K_plus:(K_plus+new_dishes)]=1       
            else:
                Ztemp=np.zeros((Z.shape[0],K_plus+new_dishes))
                Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
                #Ztemp[:,K_plus:(K_plus+new_dishes)] = 0 #added later
                Ztemp[i,K_plus:(K_plus+new_dishes)] = 1

            #Ztemp=np.zeros((Z.shape[0],K_plus+new_dishes))
            #Ztemp[0:Z.shape[0],0:Z.shape[1]]=Z
            #Ztemp[i,K_plus:K_plus+new_dishes] = 1
            Z=Ztemp
            K_plus = K_plus + new_dishes

        M=np.linalg.inv((np.dot(Z[:,0:K_plus+new_dishes].T,Z[:,0:K_plus+new_dishes]) + np.dot(((sigma_X)**2/(sigma_A)**2),np.eye(K_plus+new_dishes))))
        l_curr=func.likelihood(X, Z[:,0:(K_plus+new_dishes)], M, sigma_A, sigma_X, K_plus+new_dishes, num_objects, object_dim)
        if np.random.uniform(0,1,1)<.5:
            pr_sigma_X=sigma_X-np.random.uniform(0,1,1)/20
        else:
            pr_sigma_X=sigma_X+np.random.uniform(0,1,1)/20

        M=np.linalg.inv((np.dot(Z[:,0:K_plus+new_dishes].T,Z[:,0:K_plus+new_dishes]) + np.dot(((pr_sigma_X[0])**2/(sigma_A)**2),np.eye(K_plus+new_dishes))))
        l_new_X=func.likelihood(X, Z[:,0:(K_plus+new_dishes)], M, sigma_A, pr_sigma_X[0], K_plus+new_dishes, num_objects, object_dim)
        acc_X=np.exp(min(0,l_new_X-l_curr))

        if np.random.uniform(0,1,1)<.5:
            pr_sigma_A=sigma_A-np.random.uniform(0,1,1)/20
        else:
            pr_sigma_A=sigma_A+np.random.uniform(0,1,1)/20

        M=np.linalg.inv((np.dot(Z[:,0:K_plus+new_dishes].T,Z[:,0:K_plus+new_dishes]) + np.dot(((sigma_X)**2/(pr_sigma_A[0])**2),np.eye(K_plus+new_dishes))))
        l_new_A=func.likelihood(X, Z[:,0:(K_plus+new_dishes)], M, pr_sigma_A[0], sigma_X, K_plus+new_dishes, num_objects, object_dim)
        acc_A=np.exp(min(0,l_new_A-l_curr))

        if np.random.uniform(0,1,1)<acc_X:
            sigma_X=pr_sigma_X[0]

        if np.random.uniform(0,1,1)<acc_A:
            sigma_A=pr_sigma_A[0]

        alpha = np.random.gamma(1+K_plus, 1/(1+HN))
    
    print("Hey")
    return(chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z)
#removed K_plus + new_dishes from MH

In [35]:
from __future__ import division
np.random.seed(1)

E=1000 #change to 1000
BURN_IN=0
SAMPLE_SIZE=E-BURN_IN

sigma_A=1.
sigma_X=1.
alpha=1.

t0 = time.time()
#chain_Z, chain_K, chain_sigma_A, chain_sigma_X, chain_alpha, Z=sampler(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects)
t1 = time.time()
total=t1-t0

In [36]:
%timeit sampler(X, E, BURN_IN, SAMPLE_SIZE, sigma_A, sigma_X, alpha, object_dim, num_objects)

Hey
Hey
Hey
Hey
1 loops, best of 3: 6min 5s per loop


###6) Results 

####Total time taken 

In [27]:
total

373.23421001434326

In [ ]:
plt.hist(chain_K,bins=range(10))

####Images

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
%precision 4
plt.style.use('ggplot')

In [ ]:
plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(141)
plt.pcolormesh(A[0,:].reshape(6,6),cmap=plt.cm.gray)     
plt.subplot(142)
plt.pcolormesh(A[1,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(143)
plt.pcolormesh(A[2,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(144)
plt.pcolormesh(A[3,:].reshape(6,6),cmap=plt.cm.gray)  

In [ ]:
Z=chain_Z[999,:,0:10].reshape(100,10)
sigma_X=chain_sigma_X[999]
sigma_A=chain_sigma_A[999]
A_inf=np.dot(np.dot(np.linalg.inv((np.dot(Z.T,Z)+(sigma_X/sigma_A)*np.eye(10))),Z.T),X)

In [ ]:
plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(141)
plt.pcolormesh(A_inf[3,:].reshape(6,6),cmap=plt.cm.gray)     
plt.subplot(142)
plt.pcolormesh(A_inf[0,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(143)
plt.pcolormesh(A_inf[1,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(144)
plt.pcolormesh(A_inf[2,:].reshape(6,6),cmap=plt.cm.gray)

####Trace plots

In [ ]:
plt.plot(chain_K)
print np.mean(chain_K)
np.sum(chain_K[200:999]==4)/8

In [ ]:
plt.plot(chain_alpha)
np.mean(chain_alpha)

In [ ]:
plt.plot(chain_sigma_X)
np.mean(chain_sigma_X)

In [ ]:
plt.plot(chain_sigma_A)
np.mean(chain_sigma_A)

###7) Profiling

In [ ]:
print "Total time taken for 1000 iterations of Gibbs Sampler and Metropolis Hastings is",round(total,3),"seconds" 

###8) Optimization Strategies

1) Write the likelihood function in C or Cython

2) Probably might not be much help converting full code to C because we are using numpy anyway

3) Put matrix M calculation inside the likelihood function and not do outside again and again

4) Use Griffiths and Ghahramani (2005; Equations 51 to 54) for faster matrix inverse calculation

5) Remove extra if/else statements

6) Vectorize the loop for k_i

7) Use JIT compiling